In [1]:
import pandas as pd
import html
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from database import database_manager as dbm

/home/raimuu/Fall-24/CPSC-571/covid-twitter-sentiment/alembic.ini


**Extract**

In [2]:
# read in raw data
file_path = "../../data/processed/cleanCovidTwitterData.zip"
df = pd.read_csv(file_path, compression='zip')
display(df)
table = "processed_twitter_data"
# query = """
#     SELECT * FROM "{table}"
# """
# df = dbm.query_db(query)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64,11,110,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,"Iconoclast, cat person, soccer fan, textile & ...",07-02-2015 07:24,2321,3236,264351,False,18-08-2020 12:30,"MSNBC Well, lets qualify that: would anyone of...",['CovidVaccine'],Twitter for iPhone,False
2,Dr. Joseph Santoro,"Washington, DC 20009","Neuro PhD, #Innovator, #Technologist, #Startup...",17-01-2009 21:10,19091,20986,128119,False,18-08-2020 12:15,"Most countries, without the ability to make Va...",['Vaccines'],Hootsuite Inc.,False
3,VUMC OAP,"Nashville, TN","Office of Advanced Practice, Vanderbilt Univer...",16-03-2017 20:22,282,96,788,False,18-08-2020 11:57,DNA zooms up charts in 1st week; hear vaccine...,"['DNA', 'vaccines', 'pandemic', 'COVID19', 'Co...",Twitter Web App,False
4,Mohammadali Naseri,TEHRAN,MSc in Numerical Analysis from the Kharazmi Un...,02-02-2018 17:20,4,88,90,False,18-08-2020 11:04,its over?! Covid19Millionares covid19 corona...,"['Covid19Millionares', 'covid19', 'corona', 'C...",Twitter for iPhone,False


**Transform**

In [3]:
# drop all rows containing null
def unescape_html(text): 
    return html.unescape(text)

df_clean = df.dropna().copy()
df_clean.loc[:, 'text'] = (
    df_clean['text']
    .str.replace(r'http\S+|www.\S+', '', regex=True) # remove links
    .str.replace(r'[\n\t]', ' ', regex=True) # remove \t\n
    .apply(unescape_html) # escape html formatting
    .str.replace(r'[^\w\s.,!?;:\-()\'"/]', '', regex=True) # remove non-alphanumeric characters
)

# convert columns to integers
df_clean['user_friends'] = pd.to_numeric(df_clean['user_friends']).replace({False: 0, True: 1}).astype(int)
df_clean['user_followers'] = pd.to_numeric(df_clean['user_followers']).astype(int)
df_clean['user_favourites'] = pd.to_numeric(df_clean['user_favourites']).astype(int)
display(df_clean)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64,11,110,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,"Iconoclast, cat person, soccer fan, textile & ...",07-02-2015 07:24,2321,3236,264351,False,18-08-2020 12:30,"MSNBC Well, lets qualify that: would anyone of...",['CovidVaccine'],Twitter for iPhone,False
2,Dr. Joseph Santoro,"Washington, DC 20009","Neuro PhD, #Innovator, #Technologist, #Startup...",17-01-2009 21:10,19091,20986,128119,False,18-08-2020 12:15,"Most countries, without the ability to make Va...",['Vaccines'],Hootsuite Inc.,False
3,VUMC OAP,"Nashville, TN","Office of Advanced Practice, Vanderbilt Univer...",16-03-2017 20:22,282,96,788,False,18-08-2020 11:57,DNA zooms up charts in 1st week; hear vaccine...,"['DNA', 'vaccines', 'pandemic', 'COVID19', 'Co...",Twitter Web App,False
4,Mohammadali Naseri,TEHRAN,MSc in Numerical Analysis from the Kharazmi Un...,02-02-2018 17:20,4,88,90,False,18-08-2020 11:04,its over?! Covid19Millionares covid19 corona...,"['Covid19Millionares', 'covid19', 'corona', 'C...",Twitter for iPhone,False


In [4]:
print(df_clean['text'].head())

0    Australia to Manufacture Covid-19 Vaccine and ...
1    MSNBC Well, lets qualify that: would anyone of...
2    Most countries, without the ability to make Va...
3    DNA  zooms up charts in 1st week; hear vaccine...
4    its over?!  Covid19Millionares covid19  corona...
Name: text, dtype: object


**Load**

In [7]:
# file_path = "../../data/processed/cleanCovidTwitterData.zip"
# df_head = df_clean.head()
# df_head.to_csv(file_path, compression='zip', index=False)

"""
Create a new table with the processed data
    dbm.create_table("table_name", dataframe)
"""

"""
Replace a new table with the processed data
    dbm.create_table("table_name", dataframe, replace=True)
"""
dbm.delete_table(table_name=table)
dbm.create_table(table_name=table, dataframe=df_clean)



processed_twitter_data table dropped


InvalidRequestError: Table 'processed_twitter_data' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.